In [1]:
# STARTING_URL - has to lead to the first page of archived results!
STARTING_URL = "https://www.oddsportal.com/baseball/usa/mlb/results/"


#PATH TO BROWSER
PATH_TO_BROWSER = "/Users/marinbelaid/Documents/Chromedriver"
DATABASE_NAME = "ODDSPORTAL_DATABASE.db"

from selenium import webdriver  
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
from selenium.common.exceptions import StaleElementReferenceException
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.common.keys import Keys
import time
import pprint
import os
import sys
from datetime import datetime
import pickle
import json
from lxml import html 
from openpyxl import Workbook 
import sqlite3

TIME_PAUSE = 1.0 

In [2]:
def wait_by_xpath(xp, how_long_to_wait): # xp is string, how_long_to_wait float - the number of seconds to wait
    try:
        WebDriverWait(driver, how_long_to_wait).until(EC.presence_of_element_located((By.XPATH, xp)) )
        time.sleep(TIME_PAUSE)
        return 1 # success
    except TimeoutException:
        print ("Too much time has passed while waiting for", xp)
        return 0 # fail

def fix_string(entry_string): # remove "\n", "\t" and double spaces
    exit_string = entry_string.replace("\n", "")
    exit_string = exit_string.replace("\t", "")
    exit_string = exit_string.replace("\r", "")
    while "  " in exit_string:
        exit_string = exit_string.replace("  ", " ")
    if len(exit_string) > 0: # remove first space
        if exit_string[0] == ' ':
            exit_string = exit_string[1:len(exit_string)]
    if len(exit_string) > 0: # remove last space
        if exit_string[len(exit_string)-1] == ' ':
            exit_string = exit_string[0:len(exit_string)-1]

    return exit_string


def get_table_name_from_url(source_url):
    table_name_to_return = ""
    for char in source_url.replace('https://www.oddsportal.com', ''):
        if char.isdigit() or char.isalpha():
            table_name_to_return = table_name_to_return + char
        else:
            table_name_to_return = table_name_to_return + "_"
    return table_name_to_return


def destroy_handles_and_create_new_one():
    # call it before opening an url
    initial_handles = driver.window_handles
    driver.execute_script("window.open();")
    handles_after_opening = driver.window_handles
    added_handle = []
    for handle in handles_after_opening:
        if handle in initial_handles:
            driver.switch_to.window(handle)
            driver.close()
        else:
            added_handle.append(handle)

    driver.switch_to.window(added_handle[0])
    return


def start_driver_normal():
    normal_driver = webdriver.Chrome(PATH_TO_BROWSER)
    normal_driver.maximize_window()
    return normal_driver


def start_driver_headless():
    chrome_options = Options()
    chrome_options.add_argument("--headless")
    chrome_options.add_argument("window-size=1920,1080")
    headless_driver = webdriver.Chrome(PATH_TO_BROWSER, options=chrome_options)
    return headless_driver


def write():
    write_conn = sqlite3.connect(DATABASE_NAME)
    write_cursor = write_conn.cursor()

    column_dict = {"Team 1": 1, "Team 2":2, "Score 1":3, "Score 2":4, "Winner": 5, "Date":6, "Game Type":7, "Over 7.5 Pinnacle odds": 8, "Under 7.5 Pinnacle odds":9, "URL":10}
    wbout = Workbook()
    ws = wbout.create_sheet()
    ws.title = "Sheet1"
    current_row = 1

    for header_item in column_dict:
        ws.cell(row=current_row, column=column_dict[header_item]).value = header_item
    current_row = current_row +1

    for game_to_write in write_cursor.execute("SELECT * FROM " + CURRENT_TABLE_NAME + " ORDER BY game_unix_time DESC"):
        # index 0 is urlpart, 2 is date, 4 is game type (if empty, regular game), 5 json data (can be null)
        game_data = {}
        for header_key in column_dict:
            game_data[header_key] = ''

        game_data["URL"] = 'https://www.oddsportal.com/' + game_to_write[0]
        game_data["Date"] = game_to_write[2]
        game_data["Game Type"] = game_to_write[4]

        if game_to_write[5] != None:            
            loaded_json = json.loads(game_to_write[5])
            game_data["Team 1"] = loaded_json["team1"]
            game_data["Team 2"] = loaded_json["team2"]
            game_data["Score 1"] = int(loaded_json["score1"])
            game_data["Score 2"] = int(loaded_json["score2"])
            if game_data["Score 1"] > game_data["Score 2"]:
                game_data["Winner"] = game_data["Team 1"]
            elif game_data["Score 1"] < game_data["Score 2"]:
                game_data["Winner"] = game_data["Team 2"]

            game_data["Over 7.5 Pinnacle odds"] = round(float(loaded_json["over 7.5"]), 2)
            game_data["Under 7.5 Pinnacle odds"] = round(float(loaded_json["under 7.5"]), 2)

        # in any case, write
        for key_to_write in game_data:
            try:
                ws.cell(row=current_row, column=column_dict[key_to_write]).value = game_data[key_to_write]
            except UnicodeDecodeError:
                ws.cell(row=current_row, column=column_dict[key_to_write]).value = game_data[key_to_write].encode("utf-8", errors='ignore')
            except UnicodeEncodeError:
                ws.cell(row=current_row, column=column_dict[key_to_write]).value = game_data[key_to_write].encode("utf-8", errors='ignore')
            except TypeError:
                ws.cell(row=current_row, column=column_dict[key_to_write]).value = game_data[key_to_write].encode("utf-8", errors='ignore')
            
        current_row = current_row +1
        
    wbout.remove(wbout['Sheet'])
    exceloutname = datetime.now().strftime("%d-%m-%Y %H_%M_%S") + " MLB oddsportal.xlsx"
    wbout.save(exceloutname)
    print ("Created output file:", exceloutname)
    
    write_cursor.close()
    write_conn.close()
    return

In [3]:
################################## PHASE 1
## make a database table if it doesn't exist already.
db_conn = sqlite3.connect(DATABASE_NAME)
db_cursor = db_conn.cursor()

existing_tables = {}
tables_fetcher = db_cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
for one_table in tables_fetcher:
    existing_tables[one_table[0]] = ''


CURRENT_TABLE_NAME = get_table_name_from_url(STARTING_URL)
if CURRENT_TABLE_NAME not in existing_tables:
    # try to create the table
    try:
        db_cursor.execute("CREATE TABLE IF NOT EXISTS " + CURRENT_TABLE_NAME + " (game_url TEXT NOT NULL PRIMARY KEY, game_time_and_type TEXT, game_time_string TEXT, game_unix_time REAL, game_type TEXT, game_json_data TEXT)")
    except sqlite3.OperationalError:
        print("Can't create table since name", CURRENT_TABLE_NAME, "isn't a valid table name.")
        db_cursor.close()
        db_conn.close()
        sys.exit(0)
else:
    print("Table", CURRENT_TABLE_NAME, "already exists!")

del existing_tables

driver = start_driver_headless()
print("To write scraped data, interrupt scraping with CTRL+C in Shell and type in write() and press Enter.")



## go to first page of main URL, scrape a list of games and insert url and times until an existing game is found.
games_to_insert = {} # game url is key, values will be time string and unix
current_page = 1
while 1:
    if current_page == 1:
        page_url = STARTING_URL
    else:
        page_url = STARTING_URL + "#/page/" + str(current_page) + "/"
        
    try:
        destroy_handles_and_create_new_one()
        driver.get(page_url)
        wait_for_games = wait_by_xpath("//table[@id='tournamentTable']/tbody/tr[@xeid]", 30)
        if wait_for_games == 0:
            continue
        innerHTML_main = driver.execute_script("return document.body.innerHTML")
        htmlElem_main = html.document_fromstring(innerHTML_main)
    except KeyboardInterrupt:
        print ("Manual interrupt, quit!")
        driver.quit()
        db_cursor.close()
        db_conn.close()
        sys.exit(0)
    except:
        print ("An exception happened, try again!")
        continue

    ## parse games from this page
    games_els = htmlElem_main.xpath("//table[@id='tournamentTable']/tbody/tr[@xeid]")
    if len(games_els) == 0:
        print ("No games found, try again!")
        continue
    
    for game_el in games_els:
        game_url_el = game_el.xpath("./td[@class='name table-participant']/a[@href]")
        game_url = game_url_el[0].attrib["href"] # will add this

        game_time_type_el = game_el.xpath("./preceding-sibling::tr[@class='center nob-border'][1]/th[1]")
        time_and_type = fix_string(game_time_type_el[0].text_content()) # string, need to parse this
        game_time_object = datetime.strptime(time_and_type[0:11], "%d %b %Y")
        game_type = fix_string(time_and_type[11:len(time_and_type)].replace("-", " "))
        

        if game_url not in games_to_insert:
            games_to_insert[game_url] = {"unix_time": game_time_object.timestamp(), "string_time": time_and_type[0:11], "game_type": game_type, "time_and_type": time_and_type}

    # for ecah game in games_to_insert, check if it already exists in db. if it does, break!
    print ("Currently on page", current_page)
    need_to_break = False
    for key_to_check in games_to_insert:
        existence_identificator = db_cursor.execute("SELECT EXISTS (SELECT 1 FROM " + CURRENT_TABLE_NAME + " WHERE game_url=?)", (key_to_check,)).fetchone()[0]
        if existence_identificator == 1:
            need_to_break = True
            break

    if need_to_break:
        break # because of a game that already exists in database

    # check if next page exists.
    next_page_el = htmlElem_main.xpath("//div[@id='pagination']/span[@class='active-page']/following-sibling::a[@href and @x-page]")
    if len(next_page_el) <= 2:
        break
    else:
        current_page = current_page +1

# insert or ignore for all the games inside the dictionary.
count_preinsertion = db_cursor.execute("SELECT COUNT(*) FROM " + CURRENT_TABLE_NAME).fetchone()[0]
db_cursor.executemany("INSERT OR IGNORE INTO " + CURRENT_TABLE_NAME + " (game_url, game_time_and_type, game_time_string, game_unix_time, game_type) VALUES(?,?,?,?,?)",
                      tuple([(urlkey, games_to_insert[urlkey]["time_and_type"], games_to_insert[urlkey]["string_time"], games_to_insert[urlkey]["unix_time"],
                        games_to_insert[urlkey]["game_type"]) for urlkey in games_to_insert]) )
db_conn.commit()
count_postinsertion = db_cursor.execute("SELECT COUNT(*) FROM " + CURRENT_TABLE_NAME).fetchone()[0]
print ("New games inserted:", count_postinsertion-count_preinsertion)












################################## PHASE 2
# get all games from db where data is null, scrape
scraping_fetcher = db_cursor.execute("SELECT game_url, rowid FROM " + CURRENT_TABLE_NAME + " WHERE game_json_data IS NULL ORDER BY game_unix_time DESC")
for item_to_scrape in scraping_fetcher.fetchall():
    # index 0 is partial url, 1 is row in database
    try:
        destroy_handles_and_create_new_one()
        url_to_open = "https://www.oddsportal.com/" + item_to_scrape[0] + '#over-under;1;6.50;0'
        driver.get(url_to_open)
        odds_xpath = "//table[@class='table-main detail-odds sortable']/tbody/tr/td[1]/div/a[@href and @class='name' and text()='Pinnacle']/../../following-sibling::td[1][text()='+6.5']/following-sibling::td[contains(@class, 'right odds')]"
        wait_for_odds = wait_by_xpath(odds_xpath, 10)
        if wait_for_odds == 0:
            continue
        innerHTML = driver.execute_script("return document.body.innerHTML")
        htmlElem = html.document_fromstring(innerHTML)
    except KeyboardInterrupt:
        print ("Manual interrupt, quit!")
        driver.quit()
        db_cursor.close()
        db_conn.close()
        sys.exit(0)
    except:
        continue

    ## get odds, verify which one is over, which one under. then, get teams and final result.
    odds_els = htmlElem.xpath(odds_xpath)
    if len(odds_els) != 2:
        print ("Couldn't find 2 odds elements on URL", url_to_open)
        continue

    # if still here, there are exactly 2: verify header indexes to be over, under
    header_els = htmlElem.xpath("//table[@class='table-main detail-odds sortable']/tbody/tr/td[1]/div/a[@href and @class='name' and text()='Pinnacle']/../../../../preceding-sibling::thead/tr[1]/th[3]") + \
                 htmlElem.xpath("//table[@class='table-main detail-odds sortable']/tbody/tr/td[1]/div/a[@href and @class='name' and text()='Pinnacle']/../../../../preceding-sibling::thead/tr[1]/th[4]")

    if len(header_els) != 2:
        print("Couldn't find exactly 2 header elements on URL", url_to_open)
        continue

    # if still here, check headers
    if "over" not in header_els[0].text_content().lower() or "under" not in header_els[1].text_content().lower():
        print("Headers aren't in the right order on URL", url_to_open)
        continue

    try:
        odds_over = float(odds_els[0].text_content())
        odds_under = float(odds_els[1].text_content())
    except:
        print ("Couldn't get float values of odds on URL", url_to_open)
        continue

    # if still here, odds are good! odds_els[0] has over odd and odds_els[1] has under odd. discover team names and score line
    teams_els = htmlElem.xpath("//div[@id='event-status']/p[@class='result']/strong/../../preceding-sibling::h1[last()]")
    team1 = ''
    team2 = ''
    if len(teams_els) != 0:
        team_array = teams_els[0].text_content().split("-")
        if len(team_array) == 2:
            team1 = fix_string(team_array[0])
            team2 = fix_string(team_array[1])
    if team1 == '' or team2 == '':
        print ("Couldn't parse out teams on URL", url_to_open)
        continue

    score1 = ''
    score2 = ''
    score_el = htmlElem.xpath("//div[@id='event-status']/p[@class='result']/strong")
    if len(score_el) != 0:
        score_array = score_el[0].text_content().split(":")
        if len(score_array) == 2:
            try:
                score1 = int(score_array[0])
                score2 = int(score_array[1])
            except:
                pass

    if score1 == '' or score2 == '':
        print("Couldn't parse out scores on URL", url_to_open)
        continue

    # if still here, should be good to save
    data_to_save = {"team1": team1, "team2": team2, "score1": score1, "score2": score2, "over 7.5": odds_over, "under 7.5": odds_under}
    try:
        db_cursor.execute("UPDATE " + CURRENT_TABLE_NAME + " SET game_json_data=? WHERE rowid=?", (json.dumps(data_to_save), item_to_scrape[1]) )
        db_conn.commit()
        print ("Scraped game", item_to_scrape[0])
    except KeyboardInterrupt:
        print ("Manual interrupt, quit!")
        driver.quit()
        db_cursor.close()
        db_conn.close()
        sys.exit(0)
    except:
        continue
        

   

# end of program
driver.quit()
db_cursor.close()
db_conn.close()

write() # new connection for writing.

To write scraped data, interrupt scraping with CTRL+C in Shell and type in write() and press Enter.
Currently on page 1
Currently on page 2
Currently on page 3
Currently on page 4
Currently on page 5
Currently on page 6
Currently on page 7
Currently on page 8
Currently on page 9
Currently on page 10
Currently on page 11
Currently on page 12
Currently on page 13
Currently on page 14
Currently on page 15
Currently on page 16
Currently on page 17
Currently on page 18
Currently on page 19
Currently on page 20
New games inserted: 978
Scraped game /baseball/japan/npb/yomiuri-giants-fukuoka-s-hawks-0nBUhkJc/
Scraped game /baseball/japan/npb/yomiuri-giants-fukuoka-s-hawks-MeCQgV3i/
Scraped game /baseball/japan/npb/fukuoka-s-hawks-yomiuri-giants-8hOrVoRj/
Scraped game /baseball/japan/npb/fukuoka-s-hawks-yomiuri-giants-fkKvWRBp/
Scraped game /baseball/japan/npb/yomiuri-giants-hanshin-tigers-QDKzvEC5/
Too much time has passed while waiting for //table[@class='table-main detail-odds sortable']/tbo

Scraped game /baseball/japan/npb/nippon-ham-fighters-fukuoka-s-hawks-zoogv7MF/
Scraped game /baseball/japan/npb/orix-buffaloes-rakuten-gold-eagles-rBl1xoiS/
Scraped game /baseball/japan/npb/yomiuri-giants-hiroshima-carp-jsHls9xc/
Too much time has passed while waiting for //table[@class='table-main detail-odds sortable']/tbody/tr/td[1]/div/a[@href and @class='name' and text()='Pinnacle']/../../following-sibling::td[1][text()='+7.5']/following-sibling::td[contains(@class, 'right odds')]
Scraped game /baseball/japan/npb/chunichi-dragons-hanshin-tigers-OzP2J5pj/
Too much time has passed while waiting for //table[@class='table-main detail-odds sortable']/tbody/tr/td[1]/div/a[@href and @class='name' and text()='Pinnacle']/../../following-sibling::td[1][text()='+7.5']/following-sibling::td[contains(@class, 'right odds')]
Scraped game /baseball/japan/npb/yomiuri-giants-hiroshima-carp-tCZcKoVq/
Scraped game /baseball/japan/npb/hanshin-tigers-yakult-swallows-lEfJRTxA/
Scraped game /baseball/jap

SystemExit: 0

/Users/marinbelaid/opt/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3334: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [4]:
print("\'")

'
